# Workflow for Visualization of GMSO Topology Objects
21-03-30 <br>
Cal Craven<br>
<br>
This workflow uses some additional libraries beyond the typical [MoSDeF](https://mosdef.org/) installations to provide an example for visualizing gmso topologies. The accessibility of the components of these objects allows for conversion to dicts that dash plotly uses for interactive visualizations. For more information on using interactive dash features, see this [link](https://dash.plotly.com/).  

In [2]:
# imports
import warnings 
warnings.filterwarnings('ignore')

import unyt
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_cytoscape as cyto
import dash_table
from dash.dependencies import Input, Output
import json
from jupyter_dash import JupyterDash

import mbuild as mb
import foyer 
import gmso
from gmso.external import from_parmed
from gmso.external.convert_networkx import to_networkx

import pandas_convert

warnings.filterwarnings('ignore')

/Users/calcraven/miniconda3/envs/visual37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#Use JuypterDash to detect the proxy configuration
JupyterDash.infer_jupyter_proxy_config()

There are many ways to build a molecule using `mBuild`, for more information see the [mBuild documentation](https://mbuild.mosdef.org/en/stable/). This is a simple method of converting building a structure using a smile string.

In [5]:
#Build an mBuild compound from smiles strings
smiles = 'C1=CC=C(C=C1)C=CC=O'
molecule = mb.load(smiles, smiles=True)
molecule.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Any method you can use to generate a `gmso.Topology` class object will work for the visualization. Please raise an [issue](https://github.com/mosdef-hub/gmso/issues) if a gmso topologies is not being visualized correctly. In this case, we will take the `mBuild` object `molecule` and convert it to a gmso.Topology object by converting to a parmed object through a `Foyer` xml object.

In [6]:
#Atomtype using foyer and convert to gmso through parmed
oplsaa = foyer.Forcefield(name='oplsaa')
pmd_aa = oplsaa.apply(molecule, 
                      assert_bond_params = False,
                      assert_angle_params = False,
                      assert_dihedral_params = False)
topology = from_parmed(pmd_aa)

In [7]:
#Dash Plotly representation of the molecule
app = JupyterDash(__name__)

#determines what color each node name is shown as
color_dictionary = {'"C"':"#8C92AC",'"O"':"red",'"H"':"#848482",'"N"':"blue",'"Cl"':'green'}
color_selection = []
for key,value in color_dictionary.items():
    color_selection.append({'selector': "[element = " +  str(key) + "]",
                            'style': {'background-color': str(value),'shape': 'circle'}})

#this gives a base layout of how the molecule will be shown initially.    
graph = to_networkx(topology)
layout = nx.drawing.layout.kamada_kawai_layout(graph)

#elements is the dict that holds information about what to store in each node
elements = []
nodeids = {}
for i,node in enumerate(graph.nodes):
    elements.append({'data': {'id': str(i) + ': ' + node.name, 
                               'label': node.name + '(' + str(i) + ')',
                              'element': node.name,
                              'hash': node.__hash__(),
                              'index' : i},
                     'classes': '',
                     'position': {'x': layout[node][0]*300,'y': layout[node][1]*300},
                     'size': 200})
    nodeids[node] = str(i) + ': ' + node.name

for edge in graph.edges:
    elements.append({'data': {'source': nodeids[edge[0]], 'target': nodeids[edge[1]]}, 'classes': 'red'})
       
app.layout = html.Div([
    html.P("Molecular Visualization:"),
    #This is the format of the selectable dropdown tab
    dcc.Dropdown(
        id='parameter-dropdown',
        options=[
            {'label': 'Atom Types', 'value': 'atom_type'},
            {'label': 'Bond Types', 'value': 'bond_type'},
            {'label': 'Angle Types', 'value': 'angle_type'},
            {'label': 'Dihedral Types', 'value': 'dihedral_type'}
        ],
        value='atom_type'),
    html.Div([
        #This is the format of the molecule visualization
        cyto.Cytoscape(
            id='cytoscape',
            elements= elements,
            style={'width': '500px', 'height': '500px'},
            layout={'name':'preset'},
            stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)', 'width': "50%", 'height': "50%"
                }
            }
        ] + color_selection 
        )],
    style={'width': '50%', 'display': 'inline-block', 'margin-right': '20px',
          'borderBottom': 'thin lightgrey solid',
          'borderTop': 'thin lightgrey solid',
          'backgroundColor': 'rgb(250, 250, 250)',
          'vertical-align': 'top'}
    ),    
    html.Div([
        #this is the format of the table showing the types of the gmso.Topology
        dash_table.DataTable(
            id='table-of-parameters',
            style_as_list_view=True,
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto',
                'lineHeight': '15px'
            },
            style_table={'overflowX': 'auto','overflowY': 'auto',
                        'height': '500px', 'width': '400px'},
            style_data_conditional=[
            {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(248, 248, 248)'
            }],
            style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
            }
        )
    ],
    style={'width': '45%', 'display': 'inline-block', 'padding': '0 0'}
    )
])

#Interactive components
@app.callback(
    Output('table-of-parameters', 'columns'),
    Output('table-of-parameters', 'data'),
    Input('parameter-dropdown','value'))
def update_datatable(input_value):
    if input_value == 'atom_type':
        df = demo_utils.atomtypes_to_datatables(graph,labels=None,atom_objects = True)
    elif input_value == 'bond_type':
        df = demo_utils.bondtypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'angle_type':
        df = demo_utils.angletypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'dihedral_type':
        df = demo_utils.dihedraltypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    return([{"name": i, "id": i} for i in df.columns],
          df.to_dict('records'))

@app.callback(Output('table-of-parameters', 'style_data_conditional'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    try:
        return ([{
            'if': {'filter_query': '{atom_id} = ' + str(data['hash'])},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']),
                   'column_id': ''},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) +
                                ' || {atom4_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'}

        ])
    except TypeError:
        return

#In order to run this at on a local port, replace the line below with mode = "external"
app.run_server(mode = "inline")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-7-9858c857eef3> in update_datatable(input_value='atom_type')
     98 def update_datatable(input_value):
     99     if input_value == 'atom_type':
--> 100         df = demo_utils.atomtypes_to_datatables(graph,labels=None,atom_objects = True)
        df = undefined
        global demo_utils.atomtypes_to_datatables = undefined
        global graph = <networkx.classes.graph.Graph object at 0x7f860c6d2b90>
        global labels = undefined
        global atom_objects = undefined
    101     elif input_value == 'bond_type':
    102         df = demo_utils.bondtypes_to_datatables(graph,topology,labels=None,atom_objects = True)

NameError: name 'demo_utils' is not defined



You can select nodes on the plotly cytoscape map to allow them to be highlighted in the datatable to the right. One of the nicest features of a `gmso.Topology` object is that it is both picklable, and can be read out to a `Pandas Dataframe`. For more information on using these, see this [link](https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm). Additionally, you can rearrange the molecule to be able to orient it as you see fit by clicking and dragging each node.